[Reference](https://medium.com/@sumit.ghosh/demystifying-decorators-in-python-8ae3d3f35979)

In [1]:
def hello_world():
    print('Hello world!')

In [2]:
type(hello_world)

function

In [3]:
class Hello: 
    pass 

In [4]:
type(Hello) 

type

In [5]:
type(10)

int

In [6]:
hello = hello_world
hello() 

Hello world!


In [7]:
def wrapper_function():
     def hello_world():
         print('Hello world!')
wrapper_function()

In [8]:
def higher_order(func):
     print('Received function {} as input'.format(func))
     func()
     return func

higher_order(hello_world)

Received function <function hello_world at 0x7fdd01338440> as input
Hello world!


<function __main__.hello_world>

In [9]:
def decorator_function(func): 
    def wrapper(): 
        print('Wrapper function!') 
        print('The wrapped function is: {}'.format(func))
        print('Executing wrapped function...') 
        func() 
        print('Exiting wrapper function') 
    return wrapper

In [10]:
@decorator_function 
def hello_world(): 
    print('Hello world!') 

In [11]:
hello_world()

Wrapper function!
The wrapped function is: <function hello_world at 0x7fdd01311050>
Executing wrapped function...
Hello world!
Exiting wrapper function


In [12]:
hello_world = decorator_function(hello_world)

In [13]:
def benchmark(func): 
    import time 
    def wrapper(): 
        start = time.time() 
        func() 
        end = time.time() 
        print('[*] Execution time: {} seconds.'.format(end-start))
    return wrapper 
    
@benchmark 
def fetch_webpage(): 
    import requests 
    webpage = requests.get('https://google.com') 
    fetch_webpage()

In [14]:
fetch_webpage()

RecursionError: ignored

In [16]:
def benchmark(func): 
    import time 
    def wrapper(*args, **kwargs): 
        start = time.time() 
        return_value = func(*args, **kwargs) 
        end = time.time() 
        print('[*] Execution time: {} seconds.'.format(end-start))
        return return_value 
    return wrapper 
    
@benchmark 
def fetch_webpage(url): 
    import requests 
    webpage = requests.get(url) 
    return webpage.text 
    
webpage = fetch_webpage('https://google.com') 
print(webpage)

[*] Execution time: 0.07996702194213867 seconds.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="fWp+yjpXh1nTVe3BiO32dQ==">(function(){window.google={kEI:'03hLYImBMOSMwbkPlMuO8AY',kEXPI:'0,1302433,56976,954,5105,206,2415,2389,2316,383,246,5,1354,4920,16,314,2342,4043,1116131,1233,1196428,114,116,391,328985,51223,16115,28684,9188,8384,4859,1361,9291,3025,4742,12841,4020,978,13228,2054,918,875,4192,6430,1142,13385,4521,2774,919,2277,8,2796,1593,1279,2212,530,149,1103,840,517,1466,56,4258,1447,1,3,2063,606,2023,1777,520,1704,2565,

In [17]:
def benchmark(iters):
    def actual_decorator(func):
        import time
        
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.time()
                return_value = func(*args, **kwargs)
                end = time.time()
                total = total + (end-start)
            print('[*] Average execution time: {} seconds.'.format(total/iters))
            return return_valuereturn wrapper
    return actual_decorator
    
@benchmark(iters=10)
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text
    
webpage = fetch_webpage('https://google.com')
print(webpage)

SyntaxError: ignored

In [18]:
from collections import deque

class Memoized(object):
    def __init__(self, cache_size=100):
        self.cache_size = cache_size
        self.call_args_queue = deque()
        self.call_args_to_result = {}
        
    def __call__(self, fn, *args, **kwargs):
        def new_func(*args, **kwargs):
            memoization_key = self._convert_call_arguments_to_hash(args, kwargs)
            if memoization_key not in self.call_args_to_result:
                result = fn(*args, **kwargs)
                self._update_cache_key_with_value(memoization_key, result)
                self._evict_cache_if_necessary()
            return self.call_args_to_result[memoization_key]
        return new_func
    
    def _update_cache_key_with_value(self, key, value):
        self.call_args_to_result[key] = value
        self.call_args_queue.append(key)

    def _evict_cache_if_necessary(self):
        if len(self.call_args_queue) > self.cache_size:
            oldest_key = self.call_args_queue.popleft()
            del self.call_args_to_result[oldest_key]
            
    @staticmethod
    def _convert_call_arguments_to_hash(args, kwargs):
        return hash(str(args) + str(kwargs))

@Memoized(cache_size=5)
def get_not_so_random_number_with_max(max_value):
    import random
    return random.random() * max_value